In [19]:

import requests
from bs4 import BeautifulSoup
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
def fetch_orcid_data(orcid_id):

    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    
    # Define the ORCID API endpoint
    url = f"https://pub.orcid.org/v3.0/0000-0003-4018-6113/record"
    
    # Headers for the request
    headers = {
        "Accept": "application/xml"
    }
    
    # Make the GET request to fetch the data
    response = session.get(url)

def parse_orcid_data(xml_data):
    # Parse the XML data using Beautiful Soup
    soup = BeautifulSoup(xml_data, "lxml")
    
    # Here's a simple example to get the name of the person
    given_name = soup.find('given-names').text
    family_name = soup.find('family-name').text
    
    print(f"Name: {given_name} {family_name}")

    # You can add more parsing logic based on your needs

if __name__ == "__main__":
    orcid_id = "0000-0003-4018-6113"  # Replace with your ORCID ID
    xml_data = fetch_orcid_data(orcid_id)
    parse_orcid_data(xml_data)

KeyboardInterrupt: 

In [20]:
%pip install feedparser

     ---------------------------------------- 0.0/81.1 kB ? eta -:--:--
     ----- ---------------------------------- 10.2/81.1 kB ? eta -:--:--
     -------------- ----------------------- 30.7/81.1 kB 325.1 kB/s eta 0:00:01
     ---------------------------- --------- 61.4/81.1 kB 544.7 kB/s eta 0:00:01
     -------------------------------------- 81.1/81.1 kB 566.9 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6061 sha256=c03e4491fdf6e9b0914a1a19f34ddf10ddfb6b0baf2c3ee52b5d54c50512ad20
  Stored in directory: c:\users\aspra\appdata\local\pip\cache\wheels\3b\25\2a\105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
S

In [40]:
import feedparser

def fetch_medium_articles(user_name):
    rss_url = f"https://medium.com/feed/@{user_name}"
    feed = feedparser.parse(rss_url)
    print(f"Number of articles: {len(feed.entries)}")
    articles = []
    for entry in feed.entries:
        articles.append({
            "title": entry.title,
            "url": entry.link
        })

    return articles

# Example usage:
user_name = "aspras01"  # Replace this with your Medium username
articles = fetch_medium_articles(user_name)
for article in articles:
    print(article["title"], "-", article["url"])

Number of articles: 0


In [36]:
import feedparser
from bs4 import BeautifulSoup

def fetch_medium_articles(user_name):
    rss_url = f"https://medium.com/feed/@{user_name}"
    feed = feedparser.parse(rss_url)

    # Debug print:
    print(f"Number of articles: {len(feed.entries)}")

    articles = []
    for entry in feed.entries:
        soup = BeautifulSoup(entry.summary, "html.parser")
        
        # Fetch the cover image
        img = soup.find("img")
        img_url = img["src"] if img else None

        # Fetch the subtitle (the first paragraph in the content can serve as a subtitle)
        subtitle = soup.p.text if soup.p else None
        
        articles.append({
            "title": entry.title,
            "subtitle": subtitle,
            "url": entry.link,
            "cover_image": img_url
        })

    return articles

# Example usage:
user_name = "aspras01"  # Replace this with your Medium username
articles = fetch_medium_articles(user_name)
for article in articles:
    print(f"Title: {article['title']}")
    print(f"Subtitle: {article['subtitle']}")
    print(f"URL: {article['url']}")
    print(f"Cover Image: {article['cover_image']}\n")

Number of articles: 0


In [26]:
print(articles)

[]
